# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('assets/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():

    # YOUR CODE HERE
    spam_percentage = (spam_data['target'].sum() / len(spam_data)) * 100
    return spam_percentage

answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():

    # YOUR CODE HERE
    vect = CountVectorizer().fit(X_train)
    
    vocabulary = vect.get_feature_names_out()
    
    # Find the longest token
    longest_token = max(vocabulary, key=len)
    
    return longest_token

answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():

    # YOUR CODE HERE
    cv = CountVectorizer().fit(X_train)
    X_train_vect = cv.transform(X_train)
    X_test_vect = cv.transform(X_test)
    nbclf = MultinomialNB(alpha=0.1)
    nbclf.fit(X_train_vect, y_train)
    predicted_test = nbclf.predict(X_test_vect)
    
    return roc_auc_score(y_test, predicted_test)


answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters. The transformed data will be a compressed sparse row matrix where the number of rows is the number of documents in `X_train`, the number of columns is the number of features found by the vectorizer in each document, and each value in the sparse matrix is the tf-idf value. First find the **max** tf-idf value for every feature.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf among the **max** tf-idf values?

Put these features in two series where each series is sorted by tf-idf value. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. Any entries with identical tf-ids should appear in lexigraphically increasing order by their feature name in boh series. For example, if the features "a", "b", "c" had the tf-idfs 1.0, 0.5, 1.0 in the series with the largest tf-idfs, then they should occur in the returned result in the order "a", "c", "b" with values 1.0, 1.0, 0.5.

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():

    # YOUR CODE HERE
    vect = TfidfVectorizer().fit(X_train)
    X_train_tfidf = vect.transform(X_train)
    
    max_tfidf_per_feature = X_train_tfidf.max(axis=0).toarray()[0]
    
    feature_names = np.array(vect.get_feature_names_out())
    
    sorted_indices = max_tfidf_per_feature.argsort()
    
    smallest_features = feature_names[sorted_indices[:20]]
    smallest_tfidf_values = max_tfidf_per_feature[sorted_indices[:20]]
    
    largest_features = feature_names[sorted_indices[-20:]]
    largest_tfidf_values = max_tfidf_per_feature[sorted_indices[-20:]]
    
    smallest_series = pd.Series(smallest_tfidf_values, index=smallest_features)
    largest_series = pd.Series(largest_tfidf_values, index=largest_features)
    
    smallest_series = smallest_series.sort_values()
    largest_series = largest_series.sort_values(ascending=False)
    
    return (smallest_series, largest_series)

answer_four()

(sympathetic     0.074475
 venaam          0.074475
 pudunga         0.074475
 organizer       0.074475
 psychologist    0.074475
 stylist         0.074475
 courageous      0.074475
 chef            0.074475
 determined      0.074475
 pest            0.074475
 psychiatrist    0.074475
 exterminator    0.074475
 athletic        0.074475
 listener        0.074475
 companion       0.074475
 dependable      0.074475
 aaniye          0.074475
 healer          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 yup          1.000000
 nite         1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 146tf150p    1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 dt

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [25]:
def answer_five():

    # YOUR CODE HERE
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_tfidf = vect.transform(X_train)
    
    clf = MultinomialNB(alpha=0.1)
    clf.fit(X_train_tfidf, y_train)

    X_test_tfidf = vect.transform(X_test)
    
    y_pred_prob = clf.predict_proba(X_test_tfidf)[:, 1]
    
    auc_score = roc_auc_score(y_test, y_pred_prob)
    
    return auc_score

answer_five()

0.9954968337775665

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [26]:
def answer_six():

    # YOUR CODE HERE
    avg_length_not_spam = spam_data[spam_data['target'] == 0]['text'].apply(len).mean()
    avg_length_spam = spam_data[spam_data['target'] == 1]['text'].apply(len).mean()
    
    return (avg_length_not_spam, avg_length_spam)

answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [27]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*Hint: Since probability is set to false, use the model's `decision_function` on the test data when calculating the target scores to use in roc_auc_score*

*This function should return the AUC score as a float.*

In [28]:
from sklearn.svm import SVC

def answer_seven():

    # YOUR CODE HERE
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_tfidf = vect.transform(X_train)
    
    document_lengths = X_train.apply(len)
    X_train_final = add_feature(X_train_tfidf, document_lengths)
    
    clf = SVC(C=10000, probability=True)
    clf.fit(X_train_final, y_train)
    
    X_test_tfidf = vect.transform(X_test)
    document_lengths_test = X_test.apply(len)
    X_test_final = add_feature(X_test_tfidf, document_lengths_test)
    
    y_pred_scores = clf.decision_function(X_test_final)
    
    auc_score = roc_auc_score(y_test, y_pred_scores)
    
    return auc_score

answer_seven()

0.9963202213809143

### Question 8

What is the average number of digits per document for not spam and spam documents?

*Hint: Use `\d` for digit class*

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [35]:
import re
def answer_eight():

    digit_pattern = r'\d'
    
    avg_digits_not_spam = spam_data[spam_data['target'] == 0]['text'].apply(lambda x: len(re.findall(digit_pattern, x))).mean()
    
    avg_digits_spam = spam_data[spam_data['target'] == 1]['text'].apply(lambda x: len(re.findall(digit_pattern, x))).mean()
    
    return (avg_digits_not_spam, avg_digits_spam)

answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100` and `max_iter=1000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [36]:
from sklearn.linear_model import LogisticRegression

def answer_nine():

    # YOUR CODE HERE
    digit_train = [sum(char.isnumeric() for char in x) for x in X_train]
    digit_test = [sum(char.isnumeric() for char in x) for x in X_test]
    
    tfidf = TfidfVectorizer(min_df = 5, ngram_range = (1,3)).fit(X_train)
    X_train_tf = tfidf.transform(X_train)
    X_test_tf = tfidf.transform(X_test)
    
    X_train_tf = add_feature(X_train_tf, digit_train)
    X_test_tf = add_feature(X_test_tf, digit_test)
    
    logclf = LogisticRegression(C=100, max_iter=1000).fit(X_train_tf, y_train)
    predicted = logclf.predict(X_test_tf)
    
    return roc_auc_score(y_test, predicted)

answer_nine()

0.9678709064054463

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [37]:
def answer_ten():

    # YOUR CODE HERE
    digit_pattern = r'\w' and r'\W'
    
    avg_digits_not_spam = spam_data[spam_data['target'] == 0]['text'].apply(lambda x: len(re.findall(digit_pattern, x))).mean()
    
    avg_digits_spam = spam_data[spam_data['target'] == 1]['text'].apply(lambda x: len(re.findall(digit_pattern, x))).mean()
    
    return (avg_digits_not_spam, avg_digits_spam)


answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the **first 2000 rows** of training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100 and max_iter=1000. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [39]:
def answer_eleven():

    # YOUR CODE HERE
    len_train = [len(x) for x in X_train]
    len_test = [len(x) for x in X_test]
    digit_train = [sum(char.isnumeric() for char in x) for x in X_train]
    digit_test = [sum(char.isnumeric() for char in x) for x in X_test]
    nan_train = X_train.str.count('\W')
    nan_test = X_test.str.count('\W')
    
    cv = CountVectorizer(min_df = 5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    X_train_vect = cv.transform(X_train)
    X_test_vect = cv.transform(X_test)
    
    X_train_vect = add_feature(X_train_vect, [len_train, digit_train, nan_train])
    X_test_vect = add_feature(X_test_vect, [len_test, digit_test, nan_test])
    
    logclf = LogisticRegression(C=100, max_iter=1000).fit(X_train_vect, y_train)
    predict = logclf.predict(X_test_vect)
    
    auc_score = roc_auc_score(y_test, predict)
    
    feature_names = np.array(cv.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = logclf.coef_[0].argsort()
    small_coeffs = list(feature_names[sorted_coef_index[:10]])
    large_coeffs = list(feature_names[sorted_coef_index[:-11:-1]])
    
    return (auc_score, small_coeffs, large_coeffs)

answer_eleven()

(0.9780231906694056,
 [' i', 'ca', '..', '. ', 'pe', ' go', ' m', 'if', 'us', 'go'],
 ['digit_count', 'ia', ' r', 'xt', 'ne', 'co', ' ba', ' x', '46', 'ian '])